In [1]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_MODEL = "gpt-3.5-turbo"
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [3]:
class Country(BaseModel):
    capital: str = Field(description="capital of the country")
    name: str = Field(description="name of the country")

PROMPT_COUNTRY_INFO = """
    Provide information about {country}.
    {format_instructions}
    """

In [4]:
parser = PydanticOutputParser(pydantic_object=Country)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL)
message = HumanMessagePromptTemplate.from_template(
    template=PROMPT_COUNTRY_INFO,
)

/tmp/ipykernel_39503/1746828345.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL)


In [5]:
chat_prompt = ChatPromptTemplate.from_messages([message])
chat_prompt

ChatPromptTemplate(input_variables=['country', 'format_instructions'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country', 'format_instructions'], input_types={}, partial_variables={}, template='\n    Provide information about {country}.\n    {format_instructions}\n    '), additional_kwargs={})])

In [6]:
country_name = "Chile"

In [7]:
print("Generating response...")
chat_prompt_with_values = chat_prompt.format_prompt(
    country=country_name, format_instructions=parser.get_format_instructions()
)

Generating response...


In [8]:
output = llm(chat_prompt_with_values.to_messages())
country = parser.parse(output.content)
print(f"The capital of {country.name} is {country.capital}.")

/tmp/ipykernel_39503/2875788004.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = llm(chat_prompt_with_values.to_messages())


The capital of Chile is Santiago.
